## Data Pre-Processing

## Impoart Necessary Data and Packages

In [1]:
## FUNCTIONS
# Function:
# Combine all inputs/columns into one string

def combine_col(input_data):
    import pandas as pd
    import numpy as np
    import os
    from collections import Counter
    import nltk
    import warnings
    warnings.filterwarnings("ignore")
    
    '''Combine all inputs/columns into one string.
       Returns a DataFrame'''
    input_data['combined_data'] = input_data['name'] + " " + input_data['description'] + " " + input_data['brand_category']
    return input_data
# Function:
# Tokenize string and remove stopwords

def remove_stopwords(input_data):
    '''Tokenize string.
        Returns a list.'''

    import pandas as pd
    import numpy as np
    import os
    from collections import Counter
    import nltk
    import warnings
    warnings.filterwarnings("ignore")
    
    # combine all columns - call combin_col() function
    data2 = combine_col(input_data)

    # remove_stopwords
    from nltk.corpus import stopwords
    import nltk
    import re

    regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")
    nltk_stopwords = list((set(stopwords.words('english'))))

    # remove "unknown" since in the full_data dataset, some brand_category are unknown
    nltk_stopwords.append('unknown')

    result2 = []
    for line in data2['combined_data']:
        filtered_words = []
        if isinstance(line, str):
            line = re.sub(r'\d+\+*[\- ]*[\-]*',' ',line)
            for word in regex_word_tokenize.tokenize(line):
                if word.isdigit() == False:
                    if word.lower() not in nltk_stopwords:
                        filtered_words.append(word.lower())
            result2.append(" ".join(filtered_words))
        else:
            result2.append(np.nan)
    data2['rm_sw'] = result2
    return data2

def lemmatize_word(input_data):
    import pandas as pd
    import numpy as np
    import os
    from collections import Counter
    import nltk
    import warnings
    warnings.filterwarnings("ignore")
    
    d1 = remove_stopwords(input_data)

    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords
    nltk_stopwords = list((set(stopwords.words('english'))))
    regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")
    lemmatizer = WordNetLemmatizer()

    result3 = []
    for i in range(len(d1['rm_sw'])):
        lemmatized = []
        if isinstance(d1['rm_sw'].iloc[i],str):
            for word in regex_word_tokenize.tokenize(d1['rm_sw'].iloc[i]):
                lemmatized.append(lemmatizer.lemmatize(word))
            result3.append(" ".join(lemmatized))
        else:
            result3.append(d1['rm_sw'].iloc[i])
    d1['lemmatized'] = result3
    d1['final'] = d1['lemmatized'] + " " + d1['brand']
    d1['final_list'] = d1['final'].str.split()
    return d1

In [2]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import nltk
import warnings
warnings.filterwarnings("ignore")

## DATA-PREPROCESSING
# Import data
full_data = pd.read_csv('Full+data.csv')
# Remove some columns in full+data
x = full_data.columns[14:256]
full_data = full_data.drop(x,axis=1)
# Clean/organize values in these columns
full_data['product_id'] = full_data['product_id'].str.upper()
full_data['brand'] = full_data['brand'].str.title()
full_data['name'] = full_data['name'].str.title()
full_data['description'] = full_data['description'].str.title()
full_data['brand_category'] = full_data['brand_category'].str.title()
full_data['details'] = full_data['details'].str.title()
full_data['brand_canonical_url'] = full_data['brand_canonical_url'].str.lower()
# Import extra_data.csv and standardize string in the columns
extra = pd.read_csv('extra_data.csv')
extra['product_id'] = extra['product_id'].str.upper()
extra['brand'] = extra['brand'].str.title()
extra['name'] = extra['name'].str.title()
extra['description'] = extra['description'].str.title()
extra['brand_category'] = extra['brand_category'].str.title()
extra['details'] = extra['details'].str.title()
extra['notes'] = extra['notes'].str.lower()
extra['brand_canonical_url'] = extra['brand_canonical_url'].str.lower()
# Import first part of tagged data from USC+Product+Attribute+Data+03302020.xlsx
attr1 = pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx', sheet_name = 'Data')
attr1['attribute_name'] = attr1['attribute_name'].str.lower()
attr1['attribute_value'] = attr1['attribute_value'].str.lower()
attr2 = pd.read_csv('usc_additional_tags.csv')
attr2['attribute_name'] = attr2['attribute_name'].str.lower()
attr2['attribute_value'] = attr2['attribute_value'].str.lower()
attr1['attribute_value'] = attr1['attribute_value'].str.replace(r'[^a-zA-Z0-9<]', "")
attr2['attribute_value'] = attr2['attribute_value'].str.replace(r'[^a-zA-Z0-9<]', "")
# Standardizing attribute_name in 2nd tagged doc to the same as the 1st one
key = ['additionalcolor', 'beltbucklematerial', 'beltbuckleshape', 'beltclosure',
       'beltmaterial', 'beltwidth', 'calfwidth', 'category', 'classbelts',
       'classblazerscoatsandjackets', 'classbooties', 'classboots', 'classdress',
       'classflats', 'classhandbags', 'classjumpsuitandromper',
       'classmulesandslides', 'classpantsandleggings', 'classpumpsandheels',
       'classsandals', 'classshorts', 'classskirts', 'classslippers',
       'classsneakersandathletic', 'classsunglasses', 'classwedges',
       'closureblazerscoatsandjackets', 'closurehandbag', 'closureonepiece',
       'closurepantsandleggings', 'closureshoe', 'closureshorts', 'closureskirts',
       'closuresweater', 'closuretop', 'color', 'drycleanonly', 'embellishment',
       'fit', 'gender', 'heelheight', 'heelshape', 'legstyle', 'legstylejeans',
       'lengthblazers', 'lengthblazerscoatsandjackets', 'lengthcoatsandjackets',
       'lengthjeans', 'lengthonepiece', 'lengthpantsandleggings', 'lengthshorts',
       'lengthskirts', 'lengthtop', 'material', 'materialclothing', 'materialpurse',
       'neckline', 'occasion', 'pattern', 'primarycolor', 'print', 'rise', 'risejeans',
       'shaftheight', 'sheer', 'shoewidth', 'sizing', 'sleevelength', 'strap',
       'strapmaterial', 'style', 'subcategoryaccessory',
       'subcategoryblazerscoatsandjackets', 'subcategorybottom',
       'subcategoryonepiece', 'subcategoryshoe', 'subcategorysweater',
       'subcategorysweatshirtandhoodie', 'subcategorytop', 'sunglassframematerial',
       'sweatshirtandhoodieclosure', 'toeexposure', 'toestyle', 'trend',
       'uppermaterial', 'wash']
value = ['additional color', 'beltbuckle_material', 'belt_buckle_shape', 'belt_closure',
         'belt_material', 'belt_width', 'calf_width', 'category', 'class_belts',
         'class_blazers_coats_and_jackets', 'class_booties', 'class_boots', 'class_dress',
         'class_flats', 'class_handbags', 'class_jumpsuit_and_romper',
         'class_mules_and_slides', 'class_pants_and_leggings', 'class_pumps_and_heels',
         'class_sandals', 'class_shorts', 'class_skirts', 'class_slippers',
         'class_sneakers_and_athletic', 'class_sunglasses', 'class_wedges',
         'closure_blazers_coats_and_jackets', 'closure_handbag', 'closure_onepiece',
         'closure_pants_and_leggings', 'closure_shoe', 'closure_shorts', 'closure_skirts',
         'closure_sweater', 'closure_top', 'color', 'dry_clean_only', 'embellishment',
         'fit', 'gender', 'heel_height', 'heel_shape', 'leg_style', 'leg_style_jeans',
         'length_blazers', 'length_blazers_coats_and_jackets', 'length_coats_and_jackets',
         'length_jeans', 'length_onepiece', 'length_pants_and_leggings', 'length_shorts',
         'length_skirts', 'length_top', 'material', 'material_clothing', 'material_purse',
         'neckline', 'occasion', 'pattern', 'primary_color', 'print', 'rise', 'rise_jeans',
         'shaft_height', 'sheer', 'shoe_width', 'sizing', 'sleeve_length', 'strap',
         'strapm_aterial_', 'style', 'subcategory_accessory',
         'subcategory_blazers_coats_and_jackets', 'subcategory_bottom',
         'subcategory_onepiece', 'subcategory_shoe', 'subcategory_sweater',
         'subcategory_sweatshirt_and_hoodie', 'subcategory_top', 'sunglass_frame_material',
         'sweatshirt_and_hoodie_closure', 'toe_exposure', 'toe_style', 'trend',
         'upper_material', 'wash']
update_dict = dict(zip(key,value))
attr2['attribute_name'] = attr2['attribute_name'].map(update_dict)
attr = pd.concat([attr1, attr2]).sort_values(by = 'product_id')
attr = attr.drop_duplicates(keep = 'last').sort_values(by = 'product_id')
unique_prod_id = list(set(full_data['product_id'])) + list(set(extra['product_id']))
# Select related columns 
data1 = full_data[['product_id','brand','name','description','brand_category','details']]
data2 = extra[['product_id','brand','name','description','brand_category','details']]
data = pd.concat([data1, data2])
data['brand_category'] = data['brand_category'].str.replace(r'(/|,|:)',' ')
data['brand_category'] = data['brand_category'].str.lower()
data['brand'] = data['brand'].str.title()
data['name'] = data['name'].str.title()
data['description'] = data['description'].str.title()
data['description'] = data['description'].str.replace('\n',' ')
data['details'] = data['details'].str.title()
data['details'] = data['details'].str.replace('\n',' ')
data.drop(labels = ['details'], inplace = True, axis = 1)
data = data.replace(np.nan, '', regex=True)
data = data.replace(r'(unknown|Unknown)', '', regex=True)
data.reset_index(inplace = True)
data = data.drop(labels = ['index'], axis = 1)
final = lemmatize_word(data)
final = pd.merge(left = final, right = attr, left_on = 'product_id', right_on = 'product_id', how = 'inner')
output_data = final[['product_id','final','final_list','product_color_id','attribute_name','attribute_value']]


In [3]:
# TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

unique_prod_doc = output_data['final'].drop_duplicates(keep = 'first')
corpus = list(unique_prod_doc.values)
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
training_tfidf_vectors = vectorizer.transform(list(output_data['final']))

## Predict rest of full_data.csv

In [4]:
# Build model
from sklearn import preprocessing, linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, auc, classification_report

to_predict = lemmatize_word(data)
# to_predict = to_predict[to_predict['product_id'].isin(attr['product_id'].unique())==False]
to_predict['vectorized_doc'] = list(vectorizer.transform(to_predict['final']).toarray())
# to_predict_csv = to_predict[['product_id','brand','name','description','brand_category','final']]
to_predict_df = to_predict

In [5]:
# Dry Clean
dry_clean_only = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='dry_clean_only'].drop_duplicates()
dry_clean_onlys = output_data['attribute_value'][output_data['attribute_name']=='dry_clean_only'].unique()
dry_clean_only['dry_clean_only_encode'] = 0
dry_clean_only['dry_clean_only_encode'][dry_clean_only['attribute_value']=='yes'] = 1
dry_clean_only['vectorized_doc'] = list(vectorizer.transform(dry_clean_only['final']).toarray())

data1 = dry_clean_only
X_train = np.array(data1['vectorized_doc'])
y_train = np.array(data1['dry_clean_only_encode'])

dry_clean_model = LogisticRegression(solver='liblinear', random_state=0)
dry_clean_model.fit(list(X_train), list(y_train))

predict_data = to_predict_df
dry_clean_only_prediction = dry_clean_model.predict(list(predict_data['vectorized_doc']))
to_predict_df['dry_clean_only'] = dry_clean_only_prediction
to_predict_df['dry_clean_only'] = to_predict_df['dry_clean_only'].map({0:'no',1:'yes'})
# to_predict_df.head()

In [6]:
# Category
category = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='category'].drop_duplicates()
categories = output_data['attribute_value'][output_data['attribute_name']=='category'].unique()
category['category_encode'] = ''
categories_dict = dict(zip(categories,range(0,8)))
category['category_encode'] = category['attribute_value'].map(categories_dict)
category['vectorized_doc'] = list(vectorizer.transform(category['final']).toarray())

data1 = category
X_train = np.array(data1['vectorized_doc'])
y_train = np.array(data1['category_encode'])

category_model = DecisionTreeClassifier(max_depth = 2)
category_model.fit(list(X_train), list(y_train))

predict_data = to_predict_df
category_prediction = category_model.predict(list(predict_data['vectorized_doc']))
to_predict_df['category'] = category_prediction
to_predict_df['category'] = to_predict_df['category'].map(dict(zip(range(0,8),categories)))


In [7]:
# Embellishment
embellishment_output_data = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='embellishment']
embellishments = output_data['attribute_value'][output_data['attribute_name']=='embellishment'].unique()
embellishment = embellishment_output_data[['product_id','final','attribute_name']].drop_duplicates()
for e in embellishments:
    embellishment[e] = 0
    id_with_e = embellishment_output_data['product_id'][embellishment_output_data['attribute_value']==e].unique()
    embellishment[e][embellishment['product_id'].isin(id_with_e)] = 1
embellishment['vectorized_doc'] = list(vectorizer.transform(embellishment['final']).toarray())

data1 = embellishment
for i in embellishments:
    X_train = np.array(data1['vectorized_doc'])
    y_train = np.array(data1[i])

    model = LogisticRegression(solver='liblinear', random_state=0)
    model.fit(list(X_train), list(y_train))
    
    predict_data = to_predict_df    
    embellishment_prediction = model.predict(list(predict_data['vectorized_doc']))
    to_predict_df[f'embellishment-{i}'] = embellishment_prediction
    to_predict_df[f'embellishment-{i}'] = to_predict_df[f'embellishment-{i}'].map({0:'no',1:'yes'}, )


In [8]:
# Style
style_output_data = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='style']
styles = output_data['attribute_value'][output_data['attribute_name']=='style'].unique()
style = style_output_data[['product_id','final','attribute_name']].drop_duplicates()
for e in styles:
    style[e] = 0
    id_with_e = style_output_data['product_id'][style_output_data['attribute_value']==e].unique()
    style[e][style['product_id'].isin(id_with_e)] = 1
style['vectorized_doc'] = list(vectorizer.transform(style['final']).toarray())

data1 = style
for i in styles:
    X_train = np.array(data1['vectorized_doc'])
    y_train = np.array(data1[i].values)
    
    style_model = LogisticRegression(solver='liblinear', random_state=0)
    style_model.fit(list(X_train), list(y_train))
    
    predict_data = to_predict_df
    style_prediction = style_model.predict(list(predict_data['vectorized_doc']))
    to_predict_df[f'style-{i}'] = style_prediction
    to_predict_df[f'style-{i}'] = to_predict_df[f'style-{i}'].map({0:'no',1:'yes'})


In [9]:
# Occasion
occasion_output_data = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='occasion']
occasions = output_data['attribute_value'][output_data['attribute_name']=='occasion'].unique()
occasion = occasion_output_data[['product_id','final','attribute_name']].drop_duplicates()
for e in occasions:
    occasion[e] = 0
    id_with_e = occasion_output_data['product_id'][occasion_output_data['attribute_value']==e].unique()
    occasion[e][occasion['product_id'].isin(id_with_e)] = 1
occasion['vectorized_doc'] = list(vectorizer.transform(occasion['final']).toarray())

data1 = occasion
for i in occasions:
    X_train = np.array(data1['vectorized_doc'])
    y_train = np.array(data1[i].values)
    
    ocassion_model = LogisticRegression(solver='liblinear', random_state=0)
    ocassion_model.fit(list(X_train), list(y_train))
    
    predict_data = to_predict_df
    ocassion_prediction = ocassion_model.predict(list(predict_data['vectorized_doc']))
    to_predict_df[f'occasion-{i}'] = ocassion_prediction
    to_predict_df[f'occasion-{i}'] = to_predict_df[f'occasion-{i}'].map({0:'no',1:'yes'})


## Export full_data tagged

In [10]:
final_to_predict_df = to_predict_df.drop(labels = ['combined_data','rm_sw','lemmatized',
                                                   'final','final_list','vectorized_doc'],
                                         axis = 1)
# final_to_predict_df

In [11]:
import csv

final_to_predict_df.to_csv('full_and_extra_data_tagged.csv',doublequote=True, quoting=csv.QUOTE_ALL)

## Generate a function that will take in user's input and return an prediction output 

#### Define functions to prepare user input

In [12]:
def prepare_input(name, description, brand_category):
    input_df = pd.Series(data = [name, description, brand])
    input_df = input_df.to_frame().T
    input_df.columns = ['name','description','brand']
    input_df['combined_data'] = input_df['name'] + " " + input_df['description'] + " " + input_df['brand']

    from nltk.corpus import stopwords
    import re
    regex_word_tokenize = nltk.RegexpTokenizer(r"(\w+['-]?[a-zA-Z']*[a-z]|[0-9]+-*[0-9]*)")
    nltk_stopwords = list((set(stopwords.words('english'))))
    nltk_stopwords.append('unknown')
    input_df['combined_data'] = re.sub(r'\d+\+*[\- ]*[\-]*',' ',input_df['combined_data'][0])
    filtered_words = []
    if isinstance(input_df['combined_data'][0], str):
        for word in regex_word_tokenize.tokenize(input_df['combined_data'][0]):
            if word.isdigit() == False:
                if word.lower() not in nltk_stopwords:
                    filtered_words.append(word.lower())
        result = " ".join(filtered_words)
    input_df['rm_sw'] = result

    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords
    nltk_stopwords = list((set(stopwords.words('english'))))
    lemmatizer = WordNetLemmatizer()
    input_df['final'] = lemmatizer.lemmatize((input_df['rm_sw'][0]))
    input_df['vectorized_doc'] = list(vectorizer.transform(list(input_df['final'])).toarray())
    return input_df

#### Define functions to predict user input

In [13]:
def to_predict(name, description, brand):
    # Build model
    from sklearn import preprocessing, linear_model
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier 
    from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, auc, classification_report

    to_predict_df = prepare_input(name, description, brand)
    # Dry Clean
    dry_clean_only = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='dry_clean_only'].drop_duplicates()
    dry_clean_onlys = output_data['attribute_value'][output_data['attribute_name']=='dry_clean_only'].unique()
    dry_clean_only['dry_clean_only_encode'] = 0
    dry_clean_only['dry_clean_only_encode'][dry_clean_only['attribute_value']=='yes'] = 1
    dry_clean_only['vectorized_doc'] = list(vectorizer.transform(dry_clean_only['final']).toarray())

    data1 = dry_clean_only
    X_train = np.array(data1['vectorized_doc'])
    y_train = np.array(data1['dry_clean_only_encode'])

    dry_clean_model = LogisticRegression(solver='liblinear', random_state=0)
    dry_clean_model.fit(list(X_train), list(y_train))

    predict_data = to_predict_df
    dry_clean_only_prediction = dry_clean_model.predict(list(predict_data['vectorized_doc']))
    to_predict_df['dry_clean_only'] = dry_clean_only_prediction
    to_predict_df['dry_clean_only'] = to_predict_df['dry_clean_only'].map({0:'no',1:'yes'})

    # Category
    category = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='category'].drop_duplicates()
    categories = output_data['attribute_value'][output_data['attribute_name']=='category'].unique()
    category['category_encode'] = ''
    categories_dict = dict(zip(categories,range(0,8)))
    category['category_encode'] = category['attribute_value'].map(categories_dict)
    category['vectorized_doc'] = list(vectorizer.transform(category['final']).toarray())

    data1 = category
    X_train = np.array(data1['vectorized_doc'])
    y_train = np.array(data1['category_encode'])

    category_model = DecisionTreeClassifier(max_depth = 2)
    category_model.fit(list(X_train), list(y_train))

    predict_data = to_predict_df
    category_prediction = category_model.predict(list(predict_data['vectorized_doc']))
    to_predict_df['category'] = category_prediction
    to_predict_df['category'] = to_predict_df['category'].map(dict(zip(range(0,8),categories)))

    # Embellishment
    embellishment_output_data = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='embellishment']
    embellishments = output_data['attribute_value'][output_data['attribute_name']=='embellishment'].unique()
    embellishment = embellishment_output_data[['product_id','final','attribute_name']].drop_duplicates()
    for e in embellishments:
        embellishment[e] = 0
        id_with_e = embellishment_output_data['product_id'][embellishment_output_data['attribute_value']==e].unique()
        embellishment[e][embellishment['product_id'].isin(id_with_e)] = 1
    embellishment['vectorized_doc'] = list(vectorizer.transform(embellishment['final']).toarray())

    data1 = embellishment
    for i in embellishments:
        X_train = np.array(data1['vectorized_doc'])
        y_train = np.array(data1[i])

        model = LogisticRegression(solver='liblinear', random_state=0)
        model.fit(list(X_train), list(y_train))

        predict_data = to_predict_df    
        embellishment_prediction = model.predict(list(predict_data['vectorized_doc']))
        to_predict_df[f'embellishment-{i}'] = embellishment_prediction
        to_predict_df[f'embellishment-{i}'] = to_predict_df[f'embellishment-{i}'].map({0:'no',1:'yes'}, )

    # Style
    style_output_data = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='style']
    styles = output_data['attribute_value'][output_data['attribute_name']=='style'].unique()
    style = style_output_data[['product_id','final','attribute_name']].drop_duplicates()
    for e in styles:
        style[e] = 0
        id_with_e = style_output_data['product_id'][style_output_data['attribute_value']==e].unique()
        style[e][style['product_id'].isin(id_with_e)] = 1
    style['vectorized_doc'] = list(vectorizer.transform(style['final']).toarray())

    data1 = style
    for i in styles:
        X_train = np.array(data1['vectorized_doc'])
        y_train = np.array(data1[i].values)

        style_model = LogisticRegression(solver='liblinear', random_state=0)
        style_model.fit(list(X_train), list(y_train))

        predict_data = to_predict_df
        style_prediction = style_model.predict(list(predict_data['vectorized_doc']))
        to_predict_df[f'style-{i}'] = style_prediction
        to_predict_df[f'style-{i}'] = to_predict_df[f'style-{i}'].map({0:'no',1:'yes'})

    # Occasion
    occasion_output_data = output_data[['product_id','final','attribute_name','attribute_value']][output_data['attribute_name']=='occasion']
    occasions = output_data['attribute_value'][output_data['attribute_name']=='occasion'].unique()
    occasion = occasion_output_data[['product_id','final','attribute_name']].drop_duplicates()
    for e in occasions:
        occasion[e] = 0
        id_with_e = occasion_output_data['product_id'][occasion_output_data['attribute_value']==e].unique()
        occasion[e][occasion['product_id'].isin(id_with_e)] = 1
    occasion['vectorized_doc'] = list(vectorizer.transform(occasion['final']).toarray())

    data1 = occasion
    for i in occasions:
        X_train = np.array(data1['vectorized_doc'])
        y_train = np.array(data1[i].values)

        ocassion_model = LogisticRegression(solver='liblinear', random_state=0)
        ocassion_model.fit(list(X_train), list(y_train))

        predict_data = to_predict_df
        ocassion_prediction = ocassion_model.predict(list(predict_data['vectorized_doc']))
        to_predict_df[f'occasion-{i}'] = ocassion_prediction
        to_predict_df[f'occasion-{i}'] = to_predict_df[f'occasion-{i}'].map({0:'no',1:'yes'})
    return to_predict_df.drop(labels = ['combined_data','rm_sw','final','vectorized_doc'], axis = 1)

# Please input your item information here:

In [16]:
# ENTER ITEM'S NAME, DESCRIPTION, BRAND_CATEGORY AS STRINGS
name = input("Item name (in string): ")
description =  input("Item description (in string): ")
brand_category =  input("Item brand_category (in string): ")

to_predict(name, description, brand_category)

# DEMO

In [ ]:
##### DEMO #####
# ENTER ITEM'S NAME, DESCRIPTION, BRAND_CATEGORY AS STRINGS
name = input("Item name (in string): ")
description =  input("Item description (in string): ")
brand_category =  input("Item brand_category (in string): ")

to_predict(name, description, brand_category)